In [3]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import pandas as pd


# generating a url for job position and job location
def get_url(position,location):
    position = position.replace(' ','-')
    location = location.replace(' ','-')
    template = 'https://www.jobstreet.com.my/en/job-search/{}-jobs-in-{}/'
    url = template.format(position,location)
    return url

def get_jobs(card):
    job_name = card.find('div','sx2jih0 _2j8fZ_0 sIMFL_0 _1JtWu_0').text
    anchor_tag = card.a
    job_url = 'https://www.jobstreet.com.my'+anchor_tag['href']
    company_name = card.find('span','sx2jih0 zcydq82c _18qlyvc0 _18qlyvcv _18qlyvc1 _18qlyvc8').text
    job_location = card.find('span','sx2jih0 zcydq82c zcydq8r iwjz4h0').text
    # fetching job summary
    try:
        summary = ''
        tag_text = ''
        job_summary = card.find('ul','sx2jih0 sx2jih3 h6p8rp0 h6p8rp5')
        for eachLI in job_summary("li"):
            tag_text = eachLI.text          
            summary = summary + tag_text + ','
        if ',' in summary:
            summary = summary.rstrip(',')
        else:
            summary = 'Not Available'
    except AttributeError:
        summary = 'Not Available'
    #fetching job salary
    job_salary = card.find_all('span','sx2jih0 zcydq82c _18qlyvc0 _18qlyvcv _18qlyvc3 _18qlyvc6')
    try:
        if len(job_salary)>=2:
            job_salary = job_salary[1]
            salary = job_salary.text
        else:
            salary = 'Not Available'
    except IndexError:
        salary = 'Not Available'
    # fetching job posted date
    time_tag = card.time
    post_date = time_tag.get('datetime')
    post_date = post_date.split('T')
    post_date = post_date[0]
    today = datetime.today().strftime('%Y-%m-%d')
    
    job_info = (job_name,job_url,company_name,job_location,summary,
               salary,post_date,today)
    
    return job_info

def main(position,location):
    records = []
    
    url = get_url(position,location)
    
    # generate Response 
    response = requests.get(url)
    
    # Creating BeautifulSoup Object
    soup = BeautifulSoup(response.text,'html.parser')
    
    cards = soup.find_all('div','sx2jih0 zcydq84g zcydq83g zcydq86g zcydq85g zcydq81w zcydq82o zcydq8cg zcydq8c4')
    
    for everyCard in cards:
        jobDetails = get_jobs(everyCard)
        records.append(jobDetails)
        
    # Here We Are Using Pandas Data Frame To Save All The 
    # Jobs Information In A CSV File
    
    col = ['Job_Name','Job_URL','Company_Name','Job_Location','Summary',
               'Salary','Post_Date','Today_Date']
    job_street_data = pd.DataFrame(records,columns=col)
    
    job_street_data.to_csv('D:\\JobStreet.csv')            #convert to csv file

In [4]:
# main('job key word','location')
main('java','kuala lumpur')